# Code Startup

In [285]:
import os
import imp
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import pickle as plk
import importlib.util
from IPython.display import display, Math

###############################################################
# LINUX PATH

sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/home/leonardo/Downloads/MMI-LeonardoPessôa/MMI-Multi-Mode-Interferometer-2x2')) #Current directory


###############################################################
# WINDOWS PATH

#spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
#lumapi = importlib.util.module_from_spec(spec_win)
#os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
#sys.path.append(os.path.dirname('C:/Users/Caio/Documents/GitHub/Photonics-training/Projects/MMI/Lumericalfiles/'))
# spec_win.loader.exec_module(lumapi)

lum = lumapi.MODE(filename='MMi_Lumerical_File.lms',hide=False) 


# Initial Calculations

In [286]:
# Initial Data
ns = 1.444
nf = 3.476
lambda0 = 1550e-9
pi = np.pi 
k0 = 2*pi/lambda0
Wm = 9e-6
c0 = 299792458
N = 2

In [287]:
# TE Mode

sigma = 0
We = Wm + (lambda0/pi)*((ns/nf)**(2*sigma))*(nf**2 - ns**2)**(-1/2)
print("TE: effective width =",We)

TE: effective width = 9.156040531990453e-06


In [288]:
neffapx = np.zeros(5)
for i in range(5):
    neffapx[i] = nf - ((i+1)**2*pi*lambda0)/(We**2*4*nf*k0)
    kc = (i+1)*pi/We
    print("Approximate_Neff = %4.3f" %(neffapx[i]))

Approximate_Neff = 3.475
Approximate_Neff = 3.472
Approximate_Neff = 3.467
Approximate_Neff = 3.460
Approximate_Neff = 3.450


In [289]:
Lpi_apx = 2*pi/(neffapx[0]-neffapx[2])*1/k0 
display(Math('$$L_{\pi} = %5.2f~um $$' %(1e6*Lpi_apx)))
print("Now we have the lenght for N outputs: ")
display(Math('$$\dfrac{3}{4n}L_{\pi} = %5.2f/n~um $$' %(1e6*Lpi_apx)))
print("Since the MMI we are working at has two outputs, we have: ")
display(Math('$$\dfrac{3}{4(2)}L_{\pi} = %5.2f~um $$' %(1e6*Lpi_apx/2)))

<IPython.core.display.Math object>

Now we have the lenght for N outputs: 


<IPython.core.display.Math object>

Since the MMI we are working at has two outputs, we have: 


<IPython.core.display.Math object>

In [290]:
W_wg = (1/((2*N)**(1/4)))*np.sqrt(lambda0*We/neffapx[0])
print("For this guide, the ideal thickness of the ports will be:")
display(Math('$$W_{wg} = %5.2f~um$$' %(1e6*W_wg)))

For this guide, the ideal thickness of the ports will be:


<IPython.core.display.Math object>

# Lumerical API

Initial Parameters

In [291]:
si = "core" # Material
W = 9e-6 # Y span of the box
Lpi = 96.4101e-6 # Box Width
Wwg = 1.43e-6 # Yspan of the port
z = 0.22e-6 # Z span
zports = 5e-6 # Z span of EME ports
portlen = 7.5e-6
portyspan = 0.45e-6 
EME_cells = 150
inputport = 1 # Selected input port, counting from the upper to the lower one 


Box Creation

In [292]:
lum.switchtolayout()
lum.selectall()
lum.delete()
# Creation of MMI Box
lum.addrect()
lum.set({
    "x min"    : 0,
    "x max"    : Lpi,
    "y span"   : W,
    "y"        : 0,
    "z span"   : z,
    "z"        : 0,
    "material" : si,
    "name"     : "Box"
      })

Create Imput Ports

In [293]:
for i in range(2):
    lum.addobject("linear_taper")
    lum.set({
        "name"      : "Port "+ str(i),
        "material"  : si,
        "first axis": "y",
        "y"         : W/6*((-1)**(i+1)),
        "thickness" : 0.22e-6,
        "rotation 1": 180,
        "len"       : portlen,
        "width_l"   : Wwg,
        "width_r"   : portyspan,
        "x"         : -portlen/2,
        "angle_side": 90,
        "z"         : 0
        })
    lum.addtogroup("Input Ports")
    lum.unselectall()
    
    lum.addrect()
    lum.set({
        "x min"    : -2*portlen,
        "x max"    : -portlen/2,
        "y span"   : portyspan,
        "y"        : W/6*((-1)**(i+1)),
        "z span"   : z,
        "material" : si,
        "name"     : "Port_rectangle "+ str(i),
        "z"        : 0 
      })
    lum.addtogroup("Input Ports")
    lum.unselectall()   


Create Output Ports

In [294]:
#Create Output ports 
j = 2
for i in range(2) :
    lum.addobject("linear_taper")
    lum.set({
        "name"      : "Port "+ str(i+j),
        "material"  : si,
        "first axis": "y",
        "y"         : W/6*((-1)**(i+1)),
        "thickness" : 0.22e-6,
        "len"       : portlen,
        "width_l"   : Wwg,
        "width_r"   : portyspan,
        "x"         : Lpi + portlen/2,
        "angle_side": 90,
        "z"         : 0
        })
    lum.addtogroup("Output Ports")
    lum.unselectall()
    lum.addrect()
    lum.set({
        "x min"    : Lpi+portlen,
        "x max"    : Lpi+2*portlen,
        "y span"   : portyspan,
        "y"        : W/6*((-1)**(i+1)),
        "z span"   : z,
        "material" : si,
        "name"     : "Port_rectangle "+ str(i+j),
        "z"        : 0 
     })
    lum.addtogroup("Output Ports")
    lum.unselectall()


Create EME Solver

In [295]:
lum.addeme()
lum.set("allow custom eigensolver settings",1)
lum.set("number of cell groups",3)
lum.set("group spans",np.transpose([portlen+2e-6, Lpi, portlen+2e-6]))
lum.set({"cells"               :np.transpose([25, 1, 25]),
     "subcell method"          :np.transpose([1, 0, 1]), # 0 = none,  1 = CVCS
     "modes"                   :np.transpose([10, 50, 10]),
     "x"                       :lum.getnamed("Box", "x"),
     "y"                       :0,
     "y span"                  :W+2e-6,
     "z"                       :0,
     "z span"                  :zports,
     "y min bc"                :"PML",
     "y max bc"                :"PML",
     "z min bc"                :"Symmetric",
     "z max bc"                :"PML",
     "background material"     :"SiO2 (Glass) - Palik",
     "mesh cells y"            :EME_cells,
     "mesh cells z"            :EME_cells/2,
     "display cells"           : 0
    })

Create EME ports

In [296]:
lum.select("EME::Ports::port_1")
lum.set({"use full simulation span" :0,
    "y span"                        :Wwg*1.75,
    "y"                             :lum.getnamed("Input Ports::Port 1","y"),
    "z span"                        :zports,
    "z"                             :0
})
lum.unselectall()   
for i in range(2,4):
    if(i==2):
        lum.select("EME::Ports::port_2")
        lum.set({"use full simulation span"  :0,
             "y span"                        :Wwg*1.75,
             "y"                             :lum.getnamed("Output Ports::Port "+ str(i),"y"),
             "z span"                        :zports,
             "z"                             :0
        })
        lum.unselectall()
    else :
        lum.unselectall()
        lum.addemeport()
        lum.set({"use full simulation span"  :0,
             "y span"                        :Wwg*1.75,
             "y"                             :lum.getnamed("Output Ports::Port "+str(i),"y"),
             "z span"                        :zports,
             "z"                             :0,
             "port location"                 :"right"
        })
    lum.unselectall()


Create Field Monitor

In [297]:
lum.addemeprofile()
lum.set({"x"      :lum.getnamed("EME", "x"),
     "x span"     : Lpi+2*portlen+4e-6,
     "y span"     : lum.getnamed("EME", "y span")+2e-6
    })


Create Mesh

In [298]:
for i in range(3):
    lum.addmesh()
    lum.set({"based on a structure": 1,
     "structure"           : "Port "+str(i),
     "set mesh multiplier" : 1,
     "x mesh multiplier"   : 4,
     "y mesh multiplier"   : 4,
     "z mesh multiplier"   : 8
     })

{!!}

{!!}

